In [ ]:
! python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


#word vector spcy text classification

In [ ]:
# Re-import pandas to verify the fix
import pandas as pd
df = pd.read_json("/content/News_Category_Dataset.json",lines=True)

# Display the first few rows to confirm data loading
print(df.head())

                                   short_description  \
0  She left her husband. He killed their children...   
1                           Of course it has a song.   
2  The actor and his longtime girlfriend Anna Ebe...   
3  The actor gives Dems an ass-kicking for not fi...   
4  The "Dietland" actress said using the bags is ...   

                                            headline       date  \
0  There Were 2 Mass Shootings In Texas Last Week... 2018-05-26   
1  Will Smith Joins Diplo And Nicky Jam For The 2... 2018-05-26   
2    Hugh Grant Marries For The First Time At Age 57 2018-05-26   
3  Jim Carrey Blasts 'Castrato' Adam Schiff And D... 2018-05-26   
4  Julianna Margulies Uses Donald Trump Poop Bags... 2018-05-26   

                                                link          authors  \
0  https://www.huffingtonpost.com/entry/texas-ama...  Melissa Jeltsen   
1  https://www.huffingtonpost.com/entry/will-smit...    Andy McDonald   
2  https://www.huffingtonpost.com/entry/h

In [ ]:
df.head()

,short_description,headline,date,link,authors,category
0,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,2018-05-26,https://www.huffingtonpost.com/entry/texas-ama...,Melissa Jeltsen,CRIME
1,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...,2018-05-26,https://www.huffingtonpost.com/entry/will-smit...,Andy McDonald,ENTERTAINMENT
2,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 57,2018-05-26,https://www.huffingtonpost.com/entry/hugh-gran...,Ron Dicker,ENTERTAINMENT
3,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,2018-05-26,https://www.huffingtonpost.com/entry/jim-carre...,Ron Dicker,ENTERTAINMENT
4,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...,2018-05-26,https://www.huffingtonpost.com/entry/julianna-...,Ron Dicker,ENTERTAINMENT


In [ ]:
df.category.value_counts()

,count
category,
POLITICS,32739
ENTERTAINMENT,14257
HEALTHY LIVING,6694
QUEER VOICES,4995
BUSINESS,4254
SPORTS,4167
COMEDY,3971
PARENTS,3955
BLACK VOICES,3858


we have classes mainly of 'BUSINESS', 'SPORTS', 'CRIME' and comes under Multi-class classification Problem.

In [ ]:
df_1=df[(df['category']=='BUSINESS') | (df['category']=='SPORTS') | (df['category']=='CRIME')]

In [ ]:
df_1.category.value_counts()

,count
category,
BUSINESS,4254
SPORTS,4167
CRIME,2893


In [ ]:
df_1['label_num']=df_1.category.map({'BUSINESS':0,'SPORTS':1,'CRIME':2})

<ipython-input-14-052d5dfc2c60>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['label_num']=df_1.category.map({'BUSINESS':0,'SPORTS':1,'CRIME':2})


In [ ]:
df_1.head()

,short_description,headline,date,link,authors,category,label_num
0,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,2018-05-26,https://www.huffingtonpost.com/entry/texas-ama...,Melissa Jeltsen,CRIME,2
32,"State prosecutors say almost $84,000 had been ...",Rachel Dolezal Faces Felony Charges For Welfar...,2018-05-25,https://www.huffingtonpost.com/entry/rachel-do...,Carla Herreria,CRIME,2
40,"“We thought we were going to die,"" one of the ...","Man Faces Charges After Pulling Knife, Stun Gu...",2018-05-25,https://www.huffingtonpost.com/entry/man-knife...,Jenna Amatulli,CRIME,2
42,"A male student, believed to be the suspect, ha...",2 People Injured In Indiana School Shooting,2018-05-25,https://www.huffingtonpost.com/entry/indiana-m...,Marina Fang,CRIME,2
80,“I never want to put restrictions on the speec...,Jets Chairman Christopher Johnson Won't Fine P...,2018-05-24,https://www.huffingtonpost.com/entry/jets-chri...,Ron Dicker,SPORTS,1


In [ ]:
# prompt: Using dataframe df_1: apply min function to balance the df_1 data

# Group the dataframe by 'category' and find the minimum number of samples in any category.
min_samples = df_1.groupby('category').size().min()
# Initialize an empty list to store the sampled dataframes.
sampled_dfs = []
# Iterate through each unique category.
for category in df_1['category'].unique():
    # Filter the dataframe to get data for the current category.
    category_df = df_1[df_1['category'] == category]
    # Sample 'min_samples' rows from the current category's dataframe.
    sampled_df = category_df.sample(min_samples, random_state=42) # Use random_state for reproducibility
    # Append the sampled dataframe to the list.
    sampled_dfs.append(sampled_df)
# Concatenate all the sampled dataframes into a single balanced dataframe.
df_1 = pd.concat(sampled_dfs)

In [ ]:
df_1.category.value_counts()

,count
category,
CRIME,2893
SPORTS,2893
BUSINESS,2893


#use this utility function to preprocess the text
#1. Remove the stop words
#2. Convert to base form using lemmatisation


In [ ]:
import spacy
nlp=spacy.load('en_core_web_lg')

In [ ]:
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return ' '.join(filtered_tokens)


In [ ]:
df_1['preprocess_text']= df_1['short_description'].apply(preprocess)

In [ ]:
df_1.head()

,short_description,headline,date,link,authors,category,label_num,preprocess_text
83417,Police are trying to unravel what connects the...,Shootings On Same Street Leave 2 Dead And 3 In...,2015-08-09,https://www.huffingtonpost.com/entry/shootings...,Michael McLaughlin,CRIME,2,Police try unravel connect shooting house street
56559,Another man is still missing after a mudslide ...,Body Of Missing Gravel Pit Worker Found Nearly...,2016-06-09,https://www.huffingtonpost.com/entry/body-foun...,Nina Golgowski,CRIME,2,man miss mudslide Green Brothers Gravel Pit ne...
104621,,6 Dead After Plane Crashes Into Maryland House,2014-12-08,https://www.huffingtonpost.com/entry/6-dead-ma...,Sebastian Murdock,CRIME,2,
60557,The mother of one of the boys hopes the iPhone...,"Missing Teens' Boat, Cell Phone Found Nearly A...",2016-04-24,https://www.huffingtonpost.com/entry/missing-t...,Nina Golgowski,CRIME,2,mother boy hope iPhone reveal happen
63358,Ward's lawyers had sought to halt the executio...,"Texas Executes Adam Ward, Man Who Killed City ...",2016-03-23,https://www.huffingtonpost.com/entry/texas-exe...,,CRIME,2,Ward lawyer seek halt execution argue suffer s...


In [ ]:
# Function to get embeddings for text
def get_embedding(text):
    # Process the text with spaCy
    doc = nlp(text)
    # Return the document vector (mean of token vectors)
    return doc.vector

# Apply the get_embedding function to the 'preprocess_text' column
df_1['embedding'] = df_1['preprocess_text'].apply(get_embedding)

In [ ]:
df_1.head()

,short_description,headline,date,link,authors,category,label_num,preprocess_text,embedding
83417,Police are trying to unravel what connects the...,Shootings On Same Street Leave 2 Dead And 3 In...,2015-08-09,https://www.huffingtonpost.com/entry/shootings...,Michael McLaughlin,CRIME,2,Police try unravel connect shooting house street,"[0.26197055, 0.12994628, -0.14490075, 0.017312..."
56559,Another man is still missing after a mudslide ...,Body Of Missing Gravel Pit Worker Found Nearly...,2016-06-09,https://www.huffingtonpost.com/entry/body-foun...,Nina Golgowski,CRIME,2,man miss mudslide Green Brothers Gravel Pit ne...,"[0.14497843, 0.35055116, -0.03383936, -0.17886..."
104621,,6 Dead After Plane Crashes Into Maryland House,2014-12-08,https://www.huffingtonpost.com/entry/6-dead-ma...,Sebastian Murdock,CRIME,2,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
60557,The mother of one of the boys hopes the iPhone...,"Missing Teens' Boat, Cell Phone Found Nearly A...",2016-04-24,https://www.huffingtonpost.com/entry/missing-t...,Nina Golgowski,CRIME,2,mother boy hope iPhone reveal happen,"[-0.054233003, 0.19957502, -0.31671286, 0.1822..."
63358,Ward's lawyers had sought to halt the executio...,"Texas Executes Adam Ward, Man Who Killed City ...",2016-03-23,https://www.huffingtonpost.com/entry/texas-exe...,,CRIME,2,Ward lawyer seek halt execution argue suffer s...,"[-0.251756, 0.12298, -0.10210292, 0.1005616, -..."


In [ ]:
import sklearn.model_selection as model_selection
X_train,X_test,y_train,y_test=model_selection.train_test_split(df_1.embedding.values,df_1.label_num,test_size=0.2,random_state=42, stratify=df_1.label_num)
#

In [ ]:
X_train.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
X_train_2d = np.stack(X_train)  # some of ML
X_test_2d = np.stack(X_test)

#Attempt 1 Decision tree ML model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,classification_report
#import classification_report                                           #creating a decision tree model object
model_dt=DecisionTreeClassifier()
model_dt.fit(X_train_2d,y_train)
y_pred=model_dt.predict(X_test_2d)
accuracy_score(y_test,y_pred)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.51      0.55       578
           1       0.60      0.49      0.54       579
           2       0.51      0.69      0.58       579

    accuracy                           0.56      1736
   macro avg       0.57      0.56      0.56      1736
weighted avg       0.57      0.56      0.56      1736



use spacy glove embeddings for text vectorization.
use MultinomialNB as the classifier after applying the MinMaxscaler.


#Attempt 2 Naive bayes Multinimial

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report

# Initialize and apply MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_2d)
X_test_scaled = scaler.transform(X_test_2d)

# Initialize and train MultinomialNB
model_mnb = MultinomialNB()
model_mnb.fit(X_train_scaled, y_train)

# Make predictions and evaluate
y_pred_mnb = model_mnb.predict(X_test_scaled)
accuracy_mnb = accuracy_score(y_test, y_pred_mnb)
report_mnb = classification_report(y_test, y_pred_mnb)

print("Multinomial Naive Bayes Accuracy:", accuracy_mnb)
print("Multinomial Naive Bayes Classification Report:\n", report_mnb)

Multinomial Naive Bayes Accuracy: 0.690668202764977
Multinomial Naive Bayes Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.69      0.71       578
           1       0.83      0.55      0.66       579
           2       0.60      0.83      0.69       579

    accuracy                           0.69      1736
   macro avg       0.72      0.69      0.69      1736
weighted avg       0.72      0.69      0.69      1736



#Attempt3 KNNeighbors

In [ ]:
from sklearn import neighbors
from sklearn.metrics import accuracy_score,classification_report
model_knn=neighbors.KNeighborsClassifier()
model_knn.fit(X_train_2d,y_train)
y_pred=model_knn.predict(X_test_2d)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))


0.6514976958525346
              precision    recall  f1-score   support

           0       0.52      0.92      0.67       578
           1       0.85      0.49      0.62       579
           2       0.81      0.55      0.66       579

    accuracy                           0.65      1736
   macro avg       0.73      0.65      0.65      1736
weighted avg       0.73      0.65      0.65      1736



#attempt4 RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
model_rf=RandomForestClassifier()
model_rf.fit(X_train_2d,y_train)
y_pred=model_rf.predict(X_test_2d)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))


0.7119815668202765
              precision    recall  f1-score   support

           0       0.83      0.65      0.73       578
           1       0.76      0.66      0.71       579
           2       0.61      0.82      0.70       579

    accuracy                           0.71      1736
   macro avg       0.73      0.71      0.71      1736
weighted avg       0.73      0.71      0.71      1736



#attempt 5 GradientBoosting Classification

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score,classification_report
model_gb=GradientBoostingClassifier()
model_gb.fit(X_train_2d,y_train)
y_pred=model_gb.predict(X_test_2d)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))


0.7160138248847926
              precision    recall  f1-score   support

           0       0.84      0.65      0.73       578
           1       0.78      0.66      0.71       579
           2       0.61      0.84      0.70       579

    accuracy                           0.72      1736
   macro avg       0.74      0.72      0.72      1736
weighted avg       0.74      0.72      0.72      1736



#Best model confusing metrics

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)


array([[377,  45, 156],
       [ 38, 381, 160],
       [ 33,  61, 485]])